In [11]:
%load_ext autoreload
%autoreload 2

import gtab
import matplotlib.pyplot as plt
import gensim.downloader
from gensim.similarities.fastss import FastSS
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from get_related_words import get_similar_words

In [23]:
TERM = 'Asus ROG'

In [24]:
get_similar_words(TERM, n_words=50)

Loading saved model...


{'asusrog': ['asusrog']}

In [8]:
from cdebski.get_related_words import get_synonyms
from cdebski.get_related_words import get_hyponyms
from cdebski.get_related_words import get_related_queries
from cdebski.get_related_words import get_suggested_queries

ModuleNotFoundError: No module named 'cdebski'

In [ ]:
synonyms = get_synonyms(TERM)
print('synonyms: these words mean the same as the search term')
for term in synonyms:
    print(term)

NameError: name 'get_synonyms' is not defined

In [ ]:
hyponyms = get_hyponyms(TERM)
print('hyponyms: these words are subordinate synonyms i.e. horse is a hyponym to animal')
for term in hyponyms:
    print(term)

hyponyms: these words are subordinate synonyms i.e. horse is a hyponym to animal
['coffee_substitute']
['decaffeinated_coffee', 'decaf']
['cappuccino', 'cappuccino_coffee', 'coffee_cappuccino']
['cafe_royale', 'coffee_royal']
['Turkish_coffee']
['instant_coffee']
['mocha', 'mocha_coffee']
['espresso']
['Irish_coffee']
['iced_coffee', 'ice_coffee']
['drip_coffee']
['cafe_noir', 'demitasse']
['cafe_au_lait']


In [8]:
suggestions = get_suggested_queries(TERM)
print('suggested queries: google trends suggested terms to refine search dropdown')
for term in suggestions:
    print(term)

suggested queries: google trends suggested terms to refine search dropdown
{'mid': '/m/02vqfm', 'title': 'Coffee', 'type': 'Beverage'}
{'mid': '/m/078n6m', 'title': 'Coffee table', 'type': 'Topic'}
{'mid': '/m/07xyvk', 'title': 'Coffeemaker', 'type': 'Topic'}
{'mid': '/m/0h3nq30', 'title': 'Coffee', 'type': 'Color'}
{'mid': '/m/01p1qs', 'title': 'French press', 'type': 'Topic'}


In [9]:
related = get_related_queries(TERM)
# print('related queries: google trends related queries')
# for term in suggestions:
#     print(term)

IndexError: list index out of range

In [ ]:
t = gtab.GTAB()
t.set_options(pytrends_config={"timeframe": "2020-01-01 2024-10-01"})
query_rog = t.new_query("Instant Coffee")
query_steam = t.new_query("Iced Coffee")

In [ ]:
# plot the two separate queries
plt.plot(query_rog.index, query_rog.max_ratio, label='ROG Ally')
plt.plot(query_steam.index, query_steam.max_ratio, label='Steam Deck')
plt.legend()
plt.title('Independent Google Trends Queries with the Same Scale')
plt.show()